# Modelo de Black-Scholes-Merton con Tasas Estocásticas

El modelo de Black-Scholes-Merton está dado, en la medida ajustada por riesgo, por la siguiente ecuación diferencial estocástica:

$$dS_{t}=\left(r^{d}-r^{f}\right) S_{t}dt + \sigma S_{t} dW_{t}$$

donde $r^d$ es la tasa doméstica (constante), $r^f$ es la tasa foránea (constante) y $\sigma$ es la volatilidad de retornos logarítmicos.

Vamos a modificar este modelo y permitir que ambas tasas sigan un proceso de Hull-White. Más precisamente:

$$dS_{t}=\left(r_{t}^{d}-r_{t}^{f}\right) S_{t}dt + \sigma^W S_{t} dW_t$$

$$dr_{t}^{d}=\left(\theta_{t}^{d}-\gamma^d r_{t}^d\right)dt+\sigma^d dX_{t}$$

$$dr_{t}^{f}=\left(\theta_{t}^{f}-\gamma^f r_{t}^f-\rho_{WY}\sigma^W\sigma^f\right)dt+\sigma^f dY_{t}$$

$$dW_tdX_{t}=\rho_{WX}dt,\space dW_tdY_{t}=\rho_{WY}dt,\space dX_tdY_{t}=\rho_{XY}dt$$

Las ecuaciones están escritas en la medida ajustada por riesgo de la divisa doméstica.

## Simulación del Modelo

Para simular este modelo, para cada paso de simulación, es necesario contar con tres números aleatorios $N(0,1)$ con correlación entre sí. Para obtenerlos, recurriremos a la librería **random** de **numpy** y a la matriz de Cholesky.

Dados $numfactors=3$, $numsim=100$ y $numsteps=264$ obtengamos primeramente los números $N(0,1)$ sin correlación.

In [1]:
# Se importa numpy
import numpy as np

# Se definen las dimensiones de la simulación
numfactors=3
numsim=100
numsteps=264

# Se obtienen los números aleatorios 
sim_sin_corr = np.random.randn(numsim * numfactors, numsteps)
sim_sin_corr.shape

(300, 264)

Se define ahora la matriz que contendrá los valores de los números aleatorios correlacionados.

In [2]:
# Se define la matriz
sim_con_corr = np.zeros(numsim * numfactors * numsteps).reshape(numsim * numfactors, numsteps)

# se confirma que haya quedado con la forma correcta
sim_con_corr.shape

(300, 264)

Se construyen ahora los números correlacionados. Primero se define la matriz de correlación $\Sigma$, se utilizará una correlación constante $\rho=\frac{1}{2}$. Supondremos, además, que el primer factor es el tipo de cambio, el segundo factor es la tasa doméstica y el tercer factor es la tasa foránea.

In [3]:
sigma = np.array([[1, .5, .5], [.5, 1, .5], [.5, .5, 1]])
sigma

array([[ 1. ,  0.5,  0.5],
       [ 0.5,  1. ,  0.5],
       [ 0.5,  0.5,  1. ]])

Se calcula ahora la matriz de Cholesky $C$ de $\Sigma$.

In [4]:
C = np.linalg.cholesky(sigma)
C

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.5       ,  0.8660254 ,  0.        ],
       [ 0.5       ,  0.28867513,  0.81649658]])

Ahora sí se calculan los números correlacionados.

In [5]:
for i in range(0, numsim):
    for j in range(0, numsteps):
        e = np.array([sim_sin_corr[3 * i, j], sim_sin_corr[3 * i + 1, j], sim_sin_corr[3 * i + 2, j]]).reshape(3, 1)
        e = np.dot(C, e)
        sim_con_corr[3 * i, j] = e[0, 0]
        sim_con_corr[3 * i + 1, j] = e[1, 0]
        sim_con_corr[3 * i + 2, j] = e[2, 0]
sim_con_corr

array([[-0.2766336 ,  0.77295015, -2.1840979 , ..., -0.37773144,
         0.70888607,  2.06980056],
       [-0.07887615, -0.50805891, -2.53130387, ..., -0.74809585,
         2.06096786,  0.70504756],
       [-0.35746759,  0.23652693, -1.41903322, ...,  0.06191112,
         0.36213698,  0.33005657],
       ..., 
       [-0.96926502, -0.13086369,  0.31105251, ...,  0.50293106,
        -1.04778615, -0.89385653],
       [ 0.77436304, -0.63968392, -0.49786642, ...,  1.28685845,
        -0.89116247, -0.83376418],
       [-0.28034409, -0.37446516, -1.19973155, ..., -0.0587555 ,
        -0.56652771,  0.60649217]])

Realicemos una validación calculando la correlación entre los primeros tres caminos.

In [6]:
# Extraemos los primeros tres caminos
M = sim_con_corr[:3,:]

# Verificamos
print(M.shape)

# Se calcula la matriz de correlación
np.corrcoef(M)

(3, 264)


array([[ 1.        ,  0.40650893,  0.42852849],
       [ 0.40650893,  1.        ,  0.40858391],
       [ 0.42852849,  0.40858391,  1.        ]])

Habiendo verificado, requerimos de data inicial para poder realizar la simulación. Vamos a suponer una simulación USDCLP con $S_{0}=630.00$ y $\sigma_{FX}=12%$. Para las tasas doméstica y foránea supondremos $\sigma=.01$ y $\gamma=1$. Adicionalmente, importaremos valores de curvas para poder calibrar las funciones $\theta$ de ambas tasas.

In [7]:
s0 = 630
sigma_fx = .12
sigma = .01
gamma = 1

import pandas as pd
curva_clp = pd.read_csv("./curva_2.csv")
curva_usd = pd.read_csv("./curva_3.csv")

# Las curvas vienen representadas como factores de descuento, ahora calculamos las tasas correspondientes.
curva_clp['Tasa'] = -np.log(curva_clp['Df'])/curva_clp['Plazo']
curva_usd['Tasa'] = -np.log(curva_usd['Df'])/curva_usd['Plazo']

# Se muestra la curva_clp
curva_clp

,Plazo,Df,Tasa
0,0.002740,0.999917,0.030415
1,0.005479,0.999833,0.030415
2,0.254795,0.992906,0.027941
3,0.506849,0.986198,0.027422
4,0.758904,0.979555,0.027219
5,1.005479,0.973384,0.026829
6,1.506849,0.959941,0.027131
7,2.005479,0.944978,0.028220
8,3.010959,0.912039,0.030579
9,4.008219,0.877304,0.032658


In [8]:
# Y luego la curva_usd
curva_usd

,Plazo,Df,Tasa
0,0.002740,0.999918,0.029916
1,0.005479,0.999836,0.029915
2,0.254795,0.993033,0.027441
3,0.506849,0.986447,0.026922
4,0.758904,0.979927,0.026719
5,1.005479,0.973874,0.026329
6,1.506849,0.960665,0.026631
7,2.005479,0.945926,0.027720
8,3.010959,0.913413,0.030079
9,4.008219,0.879064,0.032158


Vamos ahora a definir las funciones que interpolarán la curva y permitirán calcular las funciones $\theta$.

In [27]:
# Se definen los arrays con los plazos
plazos_clp = np.array(curva_clp['Plazo'].tolist())
plazos_usd = np.array(curva_usd['Plazo'].tolist())


#  Se definen los arrays con las tasas
tasas_clp = np.array(curva_clp['Tasa'].tolist())
tasas_usd = np.array(curva_usd['Tasa'].tolist())

import hull_white as hw
zrate_clp = hw.get_curve(plazos_clp, tasas_clp)
zrate_usd = hw.get_curve(plazos_usd, tasas_usd)
print(zrate_clp(0)[0], zrate_usd(0)[0])
r_clp0 = zrate_clp(0)[0]
r_usd0 = zrate_usd(0)[0]

(0.030415084970720565, 0.029915581892038173)


In [10]:
# Contenedor para la simulación
# Se define la matriz
sim = np.zeros(numsim * numfactors * numsteps).reshape(numsim * numfactors, numsteps)

# se confirma que haya quedado con la forma correcta
sim.shape

(300, 264)

In [24]:
# Simular
import math
import clamped_spline as csp

# Se calculan las constantes necesarias
dt = 1 / 264.0

# Para el FX
sigma_fx_raizdt = sigma_fx * math.sqrt(dt)

# Para las tasas
gamma_dt = gamma * dt
sigma_raizdt = sigma * math.sqrt(dt)

for i in range(0, numsim):
    sim[3 * i, 0] = s0
    sim[3 * i + 1, 0] = r_clp0
    sim[3 * i + 2, 0] = r_usd0
    for j in range(1, numsteps):
        e = np.array([sim_con_corr[3 * i, j], sim_con_corr[3 * i + 1, j], sim_con_corr[3 * i + 2, j]]).reshape(3, 1)
        
        sim[3 * i, j] = sim[3 * i, j - 1] * (
            1 + (sim[3 * i + 1, j - 1] - sim[3 * i + 2, j - 1]) * dt + sigma_fx_raizdt * e[0,0])
        
        sim[3 * i + 1, j] = sim[3 * i + 1, j - 1] + hw.theta(
            zrate_clp, dt * (j - 1), sigma, gamma) * dt - gamma_dt * sim[3 * i + 1, j - 1] + sigma_raizdt * e[1,0]
        
        sim[3 * i + 2, j] = sim[3 * i + 2, j - 1] + hw.theta(
            zrate_usd, dt * (j - 1), sigma, gamma) * dt - gamma_dt * sim[3 * i + 2, j - 1] + sigma_raizdt * e[2,0]  

In [26]:
sim[2,:]

array([ 0.02991558,  0.03006102,  0.02918698,  0.02872741,  0.02810003,
        0.02839337,  0.0287048 ,  0.0291173 ,  0.02954706,  0.02960896,
        0.02914464,  0.0286621 ,  0.02825323,  0.02787794,  0.02823207,
        0.02825216,  0.0282498 ,  0.02760014,  0.02887537,  0.02910035,
        0.02856651,  0.02960582,  0.03017749,  0.02931119,  0.02895045,
        0.02853231,  0.02860208,  0.0285119 ,  0.02847022,  0.0285671 ,
        0.02779884,  0.02846332,  0.0281553 ,  0.02804579,  0.0271862 ,
        0.0281766 ,  0.0283177 ,  0.0279321 ,  0.02751958,  0.02733389,
        0.02812629,  0.02830801,  0.02746054,  0.02669049,  0.02658326,
        0.02725819,  0.02636848,  0.02675482,  0.02630565,  0.026822  ,
        0.02629377,  0.02566057,  0.02541121,  0.0250606 ,  0.02463192,
        0.02503504,  0.0256959 ,  0.02546963,  0.0251639 ,  0.02532227,
        0.02570446,  0.02580154,  0.02541805,  0.02552047,  0.02547774,
        0.02482832,  0.02571252,  0.02538829,  0.02563982,  0.02